<a href="https://colab.research.google.com/github/dpnolan/taba/blob/main/TABA_Bank_data_logic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part B
# Bank telesales data analysis using Logisitic Regression

In [4]:
# Start a new R notebook in Jupyter using URL https://colab.research.google.com/notebook#create=true&language=r 

# Install R and libraries 

The input file gets deleted whenever a runtime ends, so upload the file __Bank.csv__ through the upload icon (page plus up arrow) on the left hand column into the __sample_data__ directory

In [41]:
getwd()

[1] "/content"

# Upload and import bank data

In [42]:
setwd("/content/sample_data")

In [45]:
list.files()


[1] "anscombe.json"                "Bank.csv"                    
[3] "california_housing_test.csv"  "california_housing_train.csv"
[5] "mnist_test.csv"               "mnist_train_small.csv"       
[7] "README.md"

In [46]:
rm(bank)

In [71]:
bank=read.csv('Bank.csv',header=TRUE, sep=",", row.names=NULL)
bank=data.frame(bank)

In [72]:
head(bank)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<chr>,<chr>
1,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
2,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
3,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
4,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
5,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
6,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no


In [73]:
summary(bank)

      age            job              marital           education        
 Min.   :18.00   Length:45211       Length:45211       Length:45211      
 1st Qu.:33.00   Class :character   Class :character   Class :character  
 Median :39.00   Mode  :character   Mode  :character   Mode  :character  
 Mean   :40.94                                                           
 3rd Qu.:48.00                                                           
 Max.   :95.00                                                           
   default             balance         housing              loan          
 Length:45211       Min.   : -8019   Length:45211       Length:45211      
 Class :character   1st Qu.:    72   Class :character   Class :character  
 Mode  :character   Median :   448   Mode  :character   Mode  :character  
                    Mean   :  1362                                        
                    3rd Qu.:  1428                                        
                    Max.   :1021

In [74]:
bank_clean<-bank

# Pre-processing

Note that balance is in the dataset but not in the document   
I include it in the analysis here


In [51]:
# Convert the categorical variables to factors in R
job_factor<-factor(bank$job)

In [75]:
summary(job_factor)

admin.   blue-collar  entrepreneur     housemaid    management 
         5171          9732          1487          1240          9458 
      retired self-employed      services       student    technician 
         2264          1579          4154           938          7597 
   unemployed       unknown 
         1303           288

In [76]:
dim(job_factor)

NULL

In [77]:
bank_clean<-cbind(bank_clean,job_factor)

In [78]:
marital_factor<-factor(bank$marital)
summary(marital_factor)
bank_clean<-cbind(bank_clean,marital_factor)


divorced  married   single 
    5207    27214    12790

In [79]:
education_factor<-factor(bank$education)
summary(education_factor)
bank_clean<-cbind(bank_clean,education_factor)

primary secondary  tertiary   unknown 
     6851     23202     13301      1857

In [80]:
default_factor<-factor(bank$default)
summary(default_factor)
bank_clean<-cbind(bank_clean,default_factor)

no   yes 
44396   815

In [81]:
housing_factor<-factor(bank$housing)
summary(housing_factor)
bank_clean<-cbind(bank_clean,housing_factor)

no   yes 
20081 25130

In [82]:
loan_factor<-factor(bank$loan)
summary(loan_factor)
bank_clean<-cbind(bank_clean,loan_factor)

no   yes 
37967  7244

In [83]:
# month is converted to factor but preserving the defined calendar order
month_factor<-factor(bank$month,levels=c('jan','feb','mar','apr','may',
                                         'jun','jul','aug','sep','oct','nov','dec'))

In [84]:
summary(month_factor)
bank_clean<-cbind(bank_clean,month_factor)

jan   feb   mar   apr   may   jun   jul   aug   sep   oct   nov   dec 
 1403  2649   477  2932 13766  5341  6895  6247   579   738  3970   214

In [85]:
contact_factor<-factor(bank$contact)
summary(contact_factor)
bank_clean<-cbind(bank_clean,contact_factor)

cellular telephone   unknown 
    29285      2906     13020

In [86]:
poutcome_factor<-factor(bank$poutcome)
summary(poutcome_factor)
bank_clean<-cbind(bank_clean,poutcome_factor)

failure   other success unknown 
   4901    1840    1511   36959

In [87]:
#y is the binary outcome variable
y_factor<-factor(bank$y)
summary(y_factor)
bank_clean<-cbind(bank_clean,y_factor)

no   yes 
39922  5289

 Counts
 no = 39,922  
 yes = 5,289    
 5289/(39922+5289)=  0.1169848 approx = 11.7%,    
 Similar to the original study by Moro et al. 2014   
 The classes are significantly imbalanced,    
 so simply constantly predicting y='no' will be correct about 88.3% of the time   



In [88]:
# pdays - values for -1, we assume are NA, no recorded value
bank_clean['pdays'][bank_clean['pdays'] == -1]<-NA
pdays<-bank_clean['pdays']
#How many NA and how many real values are there? 8,257 real, 36,954 NAs
length(bank_clean$pdays)-sum(is.na(bank_clean$pdays))
summary(bank_clean$pdays)

[1] 8257

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
    1.0   133.0   194.0   224.6   327.0   871.0   36954 

In [89]:
# Delete the now factorised variables from the working data i.e.  
library('dplyr')
bank_clean <- dplyr::select(bank_clean,-job,-marital,-education,-default,-housing,-loan,
                     -contact, -poutcome,-month,-day,-pdays,-y)

In [90]:
# bank clean contains the dependent variables plus y_factor, the binary dependent variable
independent_variables <- dplyr::select (bank_clean,-y_factor)

In [91]:
# Print the column names of our working data for visual inspection of the columns remaining
colnames(bank_clean)
colnames(independent_variables)

[1] "age"              "balance"          "duration"         "campaign"        
 [5] "previous"         "job_factor"       "marital_factor"   "education_factor"
 [9] "default_factor"   "housing_factor"   "loan_factor"      "month_factor"    
[13] "contact_factor"   "poutcome_factor"  "y_factor"

[1] "age"              "balance"          "duration"         "campaign"        
 [5] "previous"         "job_factor"       "marital_factor"   "education_factor"
 [9] "default_factor"   "housing_factor"   "loan_factor"      "month_factor"    
[13] "contact_factor"   "poutcome_factor"

In [92]:
# Get summary statistics for our working dataframe
summary(bank_clean)

      age           balance          duration         campaign     
 Min.   :18.00   Min.   : -8019   Min.   :   0.0   Min.   : 1.000  
 1st Qu.:33.00   1st Qu.:    72   1st Qu.: 103.0   1st Qu.: 1.000  
 Median :39.00   Median :   448   Median : 180.0   Median : 2.000  
 Mean   :40.94   Mean   :  1362   Mean   : 258.2   Mean   : 2.764  
 3rd Qu.:48.00   3rd Qu.:  1428   3rd Qu.: 319.0   3rd Qu.: 3.000  
 Max.   :95.00   Max.   :102127   Max.   :4918.0   Max.   :63.000  
                                                                   
    previous              job_factor    marital_factor   education_factor
 Min.   :  0.0000   blue-collar:9732   divorced: 5207   primary  : 6851  
 1st Qu.:  0.0000   management :9458   married :27214   secondary:23202  
 Median :  0.0000   technician :7597   single  :12790   tertiary :13301  
 Mean   :  0.5803   admin.     :5171                    unknown  : 1857  
 3rd Qu.:  0.0000   services   :4154                                     
 Max.   :275